In [11]:
import itertools
import csv

In [12]:
# ===========================
# 1) Setup Initial Standings
# ===========================
teams_initial = {
    "HKFC7B":    {"points": 71, "wins": 12, "played": 17},
    "KCC7":      {"points": 62, "wins": 13, "played": 18},
    "HKCC6":     {"points": 57, "wins": 11, "played": 17},
    "Xavier":    {"points": 52, "wins":  7, "played": 17},
    "PC2":       {"points": 54, "wins":  9, "played": 17},
    "TSC2":      {"points": 49, "wins":  8, "played": 17},
    # Note: Twister2 has played 17 (1 match left is vs HKFC7B in Week 21)
    "Twister2":  {"points": 13, "wins":  0, "played": 17}
}

# ============================
# 2) Define Remaining Matches
# ============================
matches = [
    ("HKCC6",   "PC2"),      # (D) Week 21
    ("HKFC7B",  "Twister2"), # (E) Week 21
    ("TSC2",    "Xavier"),   # (F) Week 21
]

# =================================================
# 3) Each match has 6 possible ways to distribute
#    the 'match points' (including bonus point).
# =================================================
# Format: (points_for_teamA, points_for_teamB, wins_for_teamA, wins_for_teamB)
possible_outcomes = [
    (6, 0, 1, 0),  # Team A wins 5-0 => A gets 6 pts total, B gets 0
    (5, 1, 1, 0),  # Team A wins 4-1
    (4, 2, 1, 0),  # Team A wins 3-2
    (2, 4, 0, 1),  # Team B wins 3-2
    (1, 5, 0, 1),  # Team B wins 4-1
    (0, 6, 0, 1),  # Team B wins 5-0
]

# A parallel list that describes the rubber scores for each outcome:
# Note: Index 0 => (6,0,1,0) => A wins 5-0
#       Index 1 => (5,1,1,0) => A wins 4-1
#       Index 2 => (4,2,1,0) => A wins 3-2
#       Index 3 => (2,4,0,1) => B wins 3-2
#       Index 4 => (1,5,0,1) => B wins 4-1
#       Index 5 => (0,6,0,1) => B wins 5-0
result_labels = [
    "5-0", "4-1", "3-2", "2-3", "1-4", "0-5"
]

# -------------------------------------------------------
# 4) We'll write only the permutations where HKCC6 ends
#    outside the top 4 into a CSV file.
# -------------------------------------------------------
outfile = "hkcc6_outside_top4_permutations.csv"

with open(outfile, mode="w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    
    # --------------------------------------
    # Write a header row for clarity
    # --------------------------------------
    columns = [
        "PermIndex",
        # Instead of "OutcomeIndex", we make columns for the result strings:
        "HKCC6_vs_PC2_Result",
        "HKFC7B_vs_Twister2_Result",
        "TSC2_vs_Xavier_Result",
        "HKCC6_Rank"
    ]
    
    # Then final points & wins columns for each team
    for team in teams_initial.keys():
        columns.append(f"{team}_Points")
        columns.append(f"{team}_Wins")
    
    writer.writerow(columns)

    total_permutations = 3**6
    count_outside_top4 = 0
    
    for perm_index, combo in enumerate(itertools.product(range(6), repeat=3)):
        # Make a fresh copy of the initial table
        temp = {
            tm: {
                "points": teams_initial[tm]["points"],
                "wins":   teams_initial[tm]["wins"]
            } 
            for tm in teams_initial
        }
        
        # Apply each of the 6 match outcomes to 'temp'
        for match_idx, (teamA, teamB) in enumerate(matches):
            outcome_idx = combo[match_idx]
            pA, pB, wA, wB = possible_outcomes[outcome_idx]
            
            temp[teamA]["points"] += pA
            temp[teamB]["points"] += pB
            temp[teamA]["wins"]   += wA
            temp[teamB]["wins"]   += wB
        
        # Sort teams based on (points desc, wins desc) to get final rankings
        sorted_teams = sorted(
            temp.keys(),
            key=lambda t: (temp[t]["points"], temp[t]["wins"]),
            reverse=True
        )
        
        # Where did HKCC6 finish?
        hkcc6_final_rank = sorted_teams.index("HKCC6") + 1  # 1-based rank
        
        # If HKCC6 is 5th or below, we record this permutation
        if hkcc6_final_rank > 4:
            count_outside_top4 += 1
            
            # Prepare the row
            # Instead of outcome indices, we store the result label
            row = [perm_index]
            
            for match_outcome_idx in combo:
                row.append(result_labels[match_outcome_idx])
            
            row.append(hkcc6_final_rank)
            
            # Then add each team's final points and wins
            for team in teams_initial.keys():
                row.append(temp[team]["points"])
                row.append(temp[team]["wins"])
            
            writer.writerow(row)
    
    print(f"Total permutations checked: {total_permutations}")
    print(f"Permutations with HKCC6 outside top 4: {count_outside_top4}")
    print(f"Scenarios written to file: {outfile}")


Total permutations checked: 729
Permutations with HKCC6 outside top 4: 6
Scenarios written to file: hkcc6_outside_top4_permutations.csv


In [13]:
import pandas as pd

df = pd.read_csv("hkcc6_outside_top4_permutations.csv")
print(df.shape)  # how many rows (i.e., how many "fail" scenarios)
df.head()        # look at the first few

(6, 19)


,PermIndex,HKCC6_vs_PC2_Result,HKFC7B_vs_Twister2_Result,TSC2_vs_Xavier_Result,HKCC6_Rank,HKFC7B_Points,HKFC7B_Wins,KCC7_Points,KCC7_Wins,HKCC6_Points,HKCC6_Wins,Xavier_Points,Xavier_Wins,PC2_Points,PC2_Wins,TSC2_Points,TSC2_Wins,Twister2_Points,Twister2_Wins
0,185,0-5,5-0,0-5,5,77,13,62,13,57,11,58,8,60,10,49,8,13,0
1,191,0-5,4-1,0-5,5,76,13,62,13,57,11,58,8,60,10,49,8,14,0
2,197,0-5,3-2,0-5,5,75,13,62,13,57,11,58,8,60,10,49,8,15,0
3,203,0-5,2-3,0-5,5,73,12,62,13,57,11,58,8,60,10,49,8,17,1
4,209,0-5,1-4,0-5,5,72,12,62,13,57,11,58,8,60,10,49,8,18,1


In [14]:
df["HKCC6_Points"].value_counts()

HKCC6_Points
57    6
Name: count, dtype: int64

In [15]:
6/729

0.00823045267489712